In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
# This library is used to handle and analyze structured data.
# It allows reading CSV files, filtering rows/columns,
# converting strings to numbers, and performing group operations.
import pandas as pd

# Used to exit the program if invalid input is entered or errors occur.
# Helps in controlling the program flow based on error handling.
import sys

In [53]:
import pandas as pd
import sys

class VisitorsAnalyticsUtils:
    def __init__(self, file_path):
        self.file_path = file_path
        self.data = None


    def main():
        time_periods = {
            "1": "1978-1987",
            "2": "1988-1997",
            "3": "1998-2007",
            "4": "2008-2017"
        }

        region_map = {
            "1": "Asia",
            "2": "Europe",
            "3": "Others"
        }

        year_input = input("Enter time period(1: 1978-1987, 2: 1988-1997, 3: 1998-2007, 4: 2008-2017): ").strip()
        if year_input not in time_periods:
            print("Invalid time period")
            sys.exit()
        year_period = time_periods[year_input]

        region_input = input("Enter region (1: Asia, 2: Europe, 3: Others): ").strip()
        if region_input not in region_map:
            print("Invalid region")
            sys.exit()
        region = region_map[region_input]

        return year_period, region

    def loadDataFile(self):
        try:
            self.data = pd.read_csv(self.file_path, dtype=str)
            self.data.columns = self.data.columns.str.strip()
            self.data.rename(columns={self.data.columns[0]: "Month_Year"}, inplace=True)

            # Replace "na" with 0.0
            self.data.replace(to_replace=r'na', value='0.0', regex=True, inplace=True)

            # Convert all data (except Month_Year) to numeric
            for col in self.data.columns:
                if col != "Month_Year":
                    self.data[col] = self.data[col].str.replace(",", "", regex=False)
                    self.data[col] = pd.to_numeric(self.data[col], errors='coerce').fillna(0.0)

            # Extract year
            self.data["year"] = self.data["Month_Year"].str.extract(r"(\d{4})").astype(int)
            print("*** first 5 rows of data loaded ***")
            print(self.data.head(5))

        except Exception as e:
            print(f"Error loading file: {e}")
            sys.exit()

    def parseData(self, year_period, region_name):
        try:
            if self.data is None:
                print("Error: No data loaded. Please run `loadDataFile()` first.")
                return None

            start_year, end_year = map(int, year_period.split("-"))

            df_filtered = self.data[(self.data["year"] >= start_year) & (self.data["year"] <= end_year)]

            regions = {
                "Asia": ['Brunei Darussalam', 'Indonesia', 'Malaysia', 'Philippines', 'Thailand', 'Viet Nam',
                         'Myanmar', 'Japan', 'Hong Kong', 'India', 'China', 'Korea, Republic Of', 'Taiwan'],
                "Europe": ['United Kingdom', 'Germany', 'France', 'Italy', 'Netherlands', 'Greece',
                           'Belgium & Luxembourg', 'Switzerland', 'Austria', 'Scandinavia', 'CIS & Eastern Europe'],
                "Others": ['USA', 'Canada', 'Australia', 'New Zealand', 'Africa']
            }

            selected_columns = ['year'] + regions.get(region_name, [])
            df_parsed = df_filtered[selected_columns].copy()

            print("\n*** Parsed Data (Regions) ***")
            print(df_parsed.info())

            print("\n*** Parsed Data (Years) ***")
            print(df_parsed["year"].describe())

            self.data = df_parsed
            return df_parsed

        except Exception as e:
            print(f"An error occurred in parseData: {e}")
            return None

    def getTop3Countries(self, df):
        try:
            df_numeric = df.drop(columns=["year"])
            total_visitors = df_numeric.sum()
            top_3 = total_visitors.sort_values(ascending=False).head(3)

            print("\n*** Top 3 countries ***")
            print(top_3)

        except Exception as e:
            print(f"Error in getTop3Countries: {e}")


In [54]:
year_period, region = VisitorsAnalyticsUtils.main()
file_path = "/content/drive/MyDrive/pge_lecia/Int_Monthly_Visitor.csv"
utils = VisitorsAnalyticsUtils(file_path)
utils.loadDataFile()

df_parsed = utils.parseData(year_period, region)
if df_parsed is not None:
    utils.getTop3Countries(df_parsed)


Enter time period(1: 1978-1987, 2: 1988-1997, 3: 1998-2007, 4: 2008-2017): 3
Enter region (1: Asia, 2: Europe, 3: Others): 3
*** first 5 rows of data loaded ***
   Month_Year  Brunei Darussalam  Indonesia  Malaysia  Philippines  Thailand  \
0   1978 Jan                 0.0        0.0       0.0          0.0       0.0   
1   1978 Feb                 0.0        0.0       0.0          0.0       0.0   
2   1978 Mar                 0.0        0.0       0.0          0.0       0.0   
3   1978 Apr                 0.0        0.0       0.0          0.0       0.0   
4   1978 May                 0.0        0.0       0.0          0.0       0.0   

   Viet Nam  Myanmar  Japan  Hong Kong  ...  Switzerland  Austria  \
0       0.0      0.0  18652       3148  ...         2185      0.0   
1       0.0      0.0  20394       5805  ...         1596      0.0   
2       0.0      0.0  20136       4510  ...         1461      0.0   
3       0.0      0.0  13508       3254  ...         1472      0.0   
4       0.0  